### GPU access and Upload libraries:

In [1]:
#GPU:

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import tensorflow as tf

seed = 1

np.random.seed(1)
tf.random.set_seed(1)

2023-04-17 23:06:55.914682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch 
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import textwrap
from pprint import pprint
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

### Download the Dataset:

In [5]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-04-17 23:07:20--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.64.80.1, 2606:4700:130:436c:6f75:6466:6c61:7265
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.64.80.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  2.11MB/s    in 2.3s    

2023-04-17 23:07:23 (2.11 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [6]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [9]:
labels = df['labels']
labels.unique() #or: set(df['labels'])

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [12]:
#pick a label:
label = 'business'
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [13]:
len(texts)

510

### choose a text from the data:

In [26]:
np.random.seed(1234)
random_num = np.random.choice(len(texts))
doc =texts[random_num]

print(textwrap.fill(doc, replace_whitespace = False, fix_sentence_endings= True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to a 10-year low following the departure
of its chief executive and two members of the board.

Paul Tellier,
who was also Bombardier's president, left the company amid an ongoing
restructuring.  Laurent Beaudoin, part of the family that controls the
Montreal-based firm, will take on the role of CEO under a newly
created management structure.  Analysts said the resignations seem to
have stemmed from a boardroom dispute.  Under Mr Tellier's tenure at
the company, which began in January 2003, plans to cut the worldwide
workforce of 75,000 by almost a third by 2006 were announced.  The
firm's snowmobile division and defence services unit were also sold
and Bombardier started the development of a new aircraft seating 110
to 135 passengers.

Mr Tellier had indicated he wanted to stay at the
world's top train maker and third largest manufacturer of civil
aircraft until the restructuring was comple

### Model Pipeline:

In [27]:
mlm = pipeline('fill-mask')

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [28]:
#wexample using the first sentence in teh article:
mlm('Bombardier chief to leave <mask>')

#As can be seen below, 

[{'score': 0.06950805336236954,
  'token': 633,
  'token_str': ' job',
  'sequence': 'Bombardier chief to leave job'},
 {'score': 0.06693065166473389,
  'token': 1470,
  'token_str': ' France',
  'sequence': 'Bombardier chief to leave France'},
 {'score': 0.052735358476638794,
  'token': 558,
  'token_str': ' office',
  'sequence': 'Bombardier chief to leave office'},
 {'score': 0.025822972878813744,
  'token': 2201,
  'token_str': ' Paris',
  'sequence': 'Bombardier chief to leave Paris'},
 {'score': 0.02136850729584694,
  'token': 896,
  'token_str': ' Canada',
  'sequence': 'Bombardier chief to leave Canada'}]

In [29]:
#now trying the second sentence:
sample_text = 'Shares in <mask> and plane-making' + \
' giant Bombardier have fallen to a 10-year low following the departure' + \
' of its chief executive and two members of the board.'

mlm(sample_text)

[{'score': 0.6640951633453369,
  'token': 11016,
  'token_str': ' Airbus',
  'sequence': 'Shares in Airbus and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.26146647334098816,
  'token': 6722,
  'token_str': ' Boeing',
  'sequence': 'Shares in Boeing and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.02363526076078415,
  'token': 15064,
  'token_str': ' aerospace',
  'sequence': 'Shares in aerospace and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.014581801369786263,
  'token': 8537,
  'token_str': ' airlines',
  'sequence': 'Shares in airlines and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of th

In [30]:
sample_text = 'Shares in train and plane-making' + \
' giant Bombardier have fallen to a 10-year low following the <mask>' + \
' of its chief executive and two members of the board.'

mlm(sample_text)

[{'score': 0.5513929724693298,
  'token': 6985,
  'token_str': ' resignation',
  'sequence': 'Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the resignation of its chief executive and two members of the board.'},
 {'score': 0.21090449392795563,
  'token': 5824,
  'token_str': ' departure',
  'sequence': 'Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'},
 {'score': 0.13041989505290985,
  'token': 25624,
  'token_str': ' departures',
  'sequence': 'Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departures of its chief executive and two members of the board.'},
 {'score': 0.03651538863778114,
  'token': 14289,
  'token_str': ' dismissal',
  'sequence': 'Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the dismissal of its chief executive and two members of

In [32]:
sample_text = 'Shares in train and plane-making' + \
' giant Bombardier have fallen to a 10-year low following the <mask>' + \
' of its chief <mask> and two members of the board.'

mlm(sample_text)

[[{'score': 0.5393301248550415,
   'token': 6985,
   'token_str': ' resignation',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the resignation of its chief<mask> and two members of the board.</s>'},
  {'score': 0.21120430529117584,
   'token': 5824,
   'token_str': ' departure',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief<mask> and two members of the board.</s>'},
  {'score': 0.1283741295337677,
   'token': 25624,
   'token_str': ' departures',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departures of its chief<mask> and two members of the board.</s>'},
  {'score': 0.0430813804268837,
   'token': 14289,
   'token_str': ' dismissal',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the dismissal of its chief<ma

In [33]:
sample_text = 'Shares in train and plane-making' + \
' giant Bombardier have fallen to a 10-year low following the departure' + \
' of its chief executive and two <mask> of the board.'

mlm(sample_text)

[[{'score': 0.6555726528167725,
   'token': 6985,
   'token_str': ' resignation',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the resignation of its chief executive and two<mask> of the board.</s>'},
  {'score': 0.14694833755493164,
   'token': 5824,
   'token_str': ' departure',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two<mask> of the board.</s>'},
  {'score': 0.08734627068042755,
   'token': 25624,
   'token_str': ' departures',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departures of its chief executive and two<mask> of the board.</s>'},
  {'score': 0.04610442742705345,
   'token': 14289,
   'token_str': ' dismissal',
   'sequence': '<s>Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the dismissal of its 

### Now, writing a function to do the article spanning for us:

In [92]:
def article_spinner(text):
    #finding the best new word:
    lst_new_words = mlm(text)    
    token_str = lst_new_words[0]['token_str']
#In case there are more than one <mask>s in the text:
#     max_score = 0
#     token_str = ''
#     for i in range(len(lst_new_words)):
#         if lst_new_words[i][0]['score'] > max_score: #0: because we know it is always the first item in each list that has the max score as it is sorted already.
#             max_score = lst_new_words[i][0]['score']
#             token_str = lst_new_words[i][0]['token_str']
    #replacing the mask with the best new word:
    index_mask = text.find('<mask>') 
    before_mask = text[:index_mask-1]
    after_mask = text[index_mask+len('<mask>'):]
    new_text = before_mask + token_str + after_mask
    return new_text 

In [93]:
sample_text = 'Shares in train and plane-making' + \
' giant Bombardier have fallen to a 10-year low following the departure' + \
' of its chief executive and two <mask> of the board.'


result_ = article_spinner(sample_text)
result_

'Shares in train and plane-making giant Bombardier have fallen to a 10-year low following the departure of its chief executive and two members of the board.'